## Coursera Capstone Project Code

1. Import all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MeanShift, estimate_bandwidth

from sklearn import metrics

import folium # map rendering library

import requests


2. Get center locations for Toronto, Toronto Downtown and Toronto Midtown

In [2]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (location, latitude, longitude)

Toronto_loc = (latitude,longitude)

#### use 2.4km radius of downtown for stations
address = 'Downtown Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (location, latitude, longitude)

Downtown_loc = (latitude,longitude)

#### 3.9km radius for stations
address = 'Midtown Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (location, latitude, longitude)

Midtown_loc = (latitude,longitude)


C:\Users\jacek\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Toronto, Ontario, Canada 43.653963 -79.387207
Downtown Jewellery Mart, 257, Yonge Street, Downtown Yonge, Old Toronto, Toronto, Ontario, M5B 1N8, Canada 43.655115 -79.380219
Minto Midtown - Quantum, Anne Johnston Courtyard, Yonge & Eglinton, Old Toronto, Toronto, Ontario, M4S 3H8, Canada 43.7051999 -79.3974155214733


3. Pull list of metro stations and their locations from Foursquare

In [3]:
CLIENT_ID = 'DXTH5G523ZNYLLR345NFCITN2RDU4WWUFOCDXFK4RGNVIHON' # your Foursquare ID
CLIENT_SECRET = 'Y3GQRX4CQF53S02J45MCCVUT01A0AZ2GYAESM2SNMZNFO0S1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

    
stations_list=[]
stations_list2=[]
LIMIT = 100
metro_category = '4bf58dd8d48988d1fd931735'  # results to contain only metro stations
radius=20000

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        metro_category,
        radius, 
        LIMIT)

# make the GET request
results = requests.get(url).json()['response']['venues']


for v in results:
    print('name:', v['name'])
    print('lat lon: ', v['location']['lat'],', ',v['location']['lng'])
    stations_list2.append((v['name'], v['location']['lat'], v['location']['lng']))

Your credentails:
CLIENT_ID: DXTH5G523ZNYLLR345NFCITN2RDU4WWUFOCDXFK4RGNVIHON
CLIENT_SECRET:Y3GQRX4CQF53S02J45MCCVUT01A0AZ2GYAESM2SNMZNFO0S1
name: Eglinton Subway Station
lat lon:  43.706715975800265 ,  -79.39829838965822
name: High Park Subway Station
lat lon:  43.65392565450594 ,  -79.46686756442625
name: Royal York Subway Station
lat lon:  43.648119362241424 ,  -79.51138052162548
name: Victoria Park Subway Station
lat lon:  43.69488264493771 ,  -79.28873509168625
name: Main Street Subway Station
lat lon:  43.68910856344984 ,  -79.30166156422199
name: Glencairn Subway Station
lat lon:  43.708872397351165 ,  -79.44080107629773
name: Union Subway Station
lat lon:  43.64560188525978 ,  -79.37979340553284
name: Union Station
lat lon:  43.645167120407564 ,  -79.38064098358154
name: Pioneer Village Subway Station
lat lon:  43.77716746512319 ,  -79.51011657714844
name: Don Mills Subway Station
lat lon:  43.775501921683805 ,  -79.3465930223465
name: Lawrence Subway Station
lat lon:  43.72584

4. Classify each station's neighbourhood: Downtown, Midtown or Suburb

In [4]:
for v in stations_list2:
    #print(v[0])
    lat_lon = (v[1],v[2])
    if geopy.distance.vincenty(Downtown_loc, lat_lon).km < 2.4:
        print(v[0], ', Downtown')
        print(geopy.distance.vincenty(Downtown_loc, lat_lon).km)
        v += ('Downtown',)
 #       print('Downtown')
    elif geopy.distance.vincenty(Midtown_loc, lat_lon).km < 3.9:
        print(v[0], ', Midtown')
        print(geopy.distance.vincenty(Midtown_loc, lat_lon).km)
        v += ('Midtown',)
 #       print('Midtown')
    else:
        print(v[0], ', Suburb')
        v += ('Suburb',)
    stations_list.append(v)

   

Eglinton Subway Station , Midtown
0.18286005214417128
High Park Subway Station , Suburb
Royal York Subway Station , Suburb
Victoria Park Subway Station , Suburb
Main Street Subway Station , Suburb
Glencairn Subway Station , Midtown
3.5205927554788667
Union Subway Station , Downtown
1.0575161227964835
Union Station , Downtown
1.1057874530801972
Pioneer Village Subway Station , Suburb
Don Mills Subway Station , Suburb
Lawrence Subway Station , Midtown
2.3329505890739433
Wellesley Subway Station , Downtown
1.1420635105669354
York University Subway Station , Suburb
Sherbourne Subway Station , Downtown
1.9303187616650037
Keele Subway Station , Suburb
Spadina Subway Station , Downtown
2.3217321106792745
Lansdowne Subway Station , Suburb
Coxwell Subway Station , Suburb
Runnymede Subway Station , Suburb
Kennedy Subway/RT Station , Suburb
Bloor-Yonge Subway Station , Downtown
1.7948163166579858
Kipling Subway Station , Suburb
Eglinton West Subway Station , Midtown
3.2054397694289674
St. George 

Create data frame with stations, location details and category

In [5]:
stat_df = pd.DataFrame(stations_list, columns=['Station Name','lat', 'lng', 'Station location'])

5. Show stations on the map

In [6]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

for name, lat, lng, loc in zip(stat_df['Station Name'], stat_df['lat'], stat_df['lng'], stat_df['Station location']):
    name = folium.Popup(name, parse_html=True)
    if loc=='Downtown':
        folium.CircleMarker(
            [float(lat), float(lng)],
            radius=5,
            popup=name,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.5,
            parse_html=False).add_to(map_toronto)
    elif loc == 'Midtown':
        folium.CircleMarker(
            [float(lat), float(lng)],
            radius=5,
            popup=name,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.5,
            parse_html=False).add_to(map_toronto)
    else:
         folium.CircleMarker(
            [float(lat), float(lng)],
            radius=5,
            popup=name,
            color='yellow',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.5,
            parse_html=False).add_to(map_toronto) 

In [7]:
map_toronto

6. Collect list of venues 400m around each station

In [9]:
venues_list=[]
LIMIT = 100
radius=400
for name, lat, lng, loc in zip(stat_df['Station Name'], stat_df['lat'], stat_df['lng'], stat_df['Station location']):
    print(name)
   
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        loc,
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])


    
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Station Name', 
                         'Station Latitude', 
                         'Station Longitude', 
                         'Station Location',
                         'Venue', 
                         'Venue Latitude', 
                         'Venue Longitude', 
                         'Venue Category']  
   

Eglinton Subway Station
High Park Subway Station
Royal York Subway Station
Victoria Park Subway Station
Main Street Subway Station
Glencairn Subway Station
Union Subway Station
Union Station
Pioneer Village Subway Station
Don Mills Subway Station
Lawrence Subway Station
Wellesley Subway Station
York University Subway Station
Sherbourne Subway Station
Keele Subway Station
Spadina Subway Station
Lansdowne Subway Station
Coxwell Subway Station
Runnymede Subway Station
Kennedy Subway/RT Station
Bloor-Yonge Subway Station
Kipling Subway Station
Eglinton West Subway Station
St. George Subway Station
Islington Subway Station
Dufferin Subway Station
Davisville Subway Station
Rosedale Subway Station
Donlands Subway Station
Greenwood Subway Station
Vaughan Metropolitan Centre Subway Station
Queen Subway Station
Bathurst Subway Station
Christie Subway Station
Jane Subway Station
Museum Subway Station
Dundas West Subway Station
St Andrew Subway Station
Pape Subway Station
Lawrence West Subway Stat

7. Remove venues that are not in scope from the list

In [10]:
df=nearby_venues[~nearby_venues['Venue Category'].isin(['Beach',
                 'Bridge',
                 'Castle',
                 'City',
                 'Community Center',
                 'Lake',
                 'Historic Site',
                 'Monument / Landmark',
                 'Park',
                 'Neighborhood',
                 'Scenic Lookout',
                 'Town',
                 'Trail',
                 'Plaza',
                 'Baseball Field',
                 'Bookstore',
                 'Train Station',
                 'Electronics Store',
                 'Hotel',
                 'Library',
                 'Tailor Shop',
                 'Hostel',
                 'Bus Station',
                 'Bus Stop',
                 'Bus Line',
                 'Light Rail Station',
                 'Clothing Store',
                 'Art Galery',
                 'Art Museum',
                 'Antique Shop',
                 'Furniture / Home Store',
                 'Metro Station',
                 'Intersection',
                 'Arts & Crafts Store',
                 'Hobby Shop',
                 'Track',
                 'Playground'])]


8. Code and group venues of the same type, inlcude Station Location category as one of attributes

In [11]:

df_onehot = pd.get_dummies(data = df, columns=['Venue Category','Station Location'], prefix="", prefix_sep="")

df_onehot['Station Name'] = df['Station Name']
df_onehot.drop(['Station Latitude','Station Longitude', 'Venue', 'Venue Latitude','Venue Longitude'], axis=1, inplace=True)

df_grouped = df_onehot.groupby('Station Name').mean().reset_index()

9. List top venues 

In [12]:
num_top_venues = 6   #Station Location category is aloways on top, so 6 overall

for hood in df_grouped['Station Name']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Station Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 16   # 15 Top venues as 1st is always Station Location

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Station Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

----Bathurst Subway Station----
                         venue  freq
0                       Suburb  1.00
1            Korean Restaurant  0.12
2                   Restaurant  0.05
3  Eastern European Restaurant  0.05
4               Ice Cream Shop  0.05
5                  Pizza Place  0.05


----Bloor-Yonge Subway Station----
                venue  freq
0            Downtown  1.00
1                Café  0.10
2         Coffee Shop  0.10
3  Italian Restaurant  0.04
4    Sushi Restaurant  0.04
5          Restaurant  0.04


----Broadview Subway Station----
            venue  freq
0          Suburb  1.00
1        Pharmacy  0.05
2     Coffee Shop  0.05
3             Spa  0.05
4  Sandwich Place  0.05
5             Pub  0.05


----Christie Subway Station----
               venue  freq
0             Suburb  1.00
1  Korean Restaurant  0.31
2        Coffee Shop  0.07
3  Indian Restaurant  0.07
4      Grocery Store  0.04
5     Ice Cream Shop  0.04


----Coxwell Subway Station----
               ve

                venue  freq
0            Downtown  1.00
1         Coffee Shop  0.12
2                Café  0.06
3          Steakhouse  0.04
4  Italian Restaurant  0.03
5                 Gym  0.03


----St Clair Subway Station----
                venue  freq
0             Midtown  1.00
1         Coffee Shop  0.10
2  Italian Restaurant  0.08
3                Café  0.06
4    Sushi Restaurant  0.06
5          Bagel Shop  0.04


----St Clair West Subway Station----
            venue  freq
0         Midtown  1.00
1    Dessert Shop  0.11
2     Pizza Place  0.11
3  Ice Cream Shop  0.11
4     Coffee Shop  0.05
5   Grocery Store  0.05


----St. George Subway Station----
                 venue  freq
0             Downtown  1.00
1          Coffee Shop  0.14
2       Cosmetics Shop  0.05
3           Restaurant  0.05
4       Sandwich Place  0.05
5  Sporting Goods Shop  0.03


----Union Station----
                venue  freq
0            Downtown  1.00
1         Coffee Shop  0.19
2          Restauran

In [13]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Station Name'] = df_grouped['Station Name']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Station Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Bathurst Subway Station,Suburb,Korean Restaurant,Restaurant,Pizza Place,Eastern European Restaurant,Ice Cream Shop,Grocery Store,Coffee Shop,Spa,Mexican Restaurant,Café,Caribbean Restaurant,Chinese Restaurant,School,Dessert Shop,Fast Food Restaurant
1,Bloor-Yonge Subway Station,Downtown,Café,Coffee Shop,Gym,Restaurant,Indian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Yoga Studio,Mediterranean Restaurant,French Restaurant,Burger Joint,Deli / Bodega
2,Broadview Subway Station,Suburb,Sandwich Place,Pharmacy,Pizza Place,Spa,Pub,Coffee Shop,Breakfast Spot,Burger Joint,Mexican Restaurant,Café,Fish & Chips Shop,Churrascaria,Falafel Restaurant,Fast Food Restaurant,Liquor Store
3,Christie Subway Station,Suburb,Korean Restaurant,Indian Restaurant,Coffee Shop,Ice Cream Shop,Japanese Restaurant,Grocery Store,Cocktail Bar,Café,Pizza Place,Ethiopian Restaurant,Mexican Restaurant,Bubble Tea Shop,Rock Climbing Spot,Gift Shop,Taco Place
4,Coxwell Subway Station,Suburb,Gastropub,Bank,Pizza Place,Pharmacy,Sandwich Place,Cocktail Bar,French Restaurant,Breakfast Spot,Bowling Alley,Gym,Bar,Burger Joint,Gym / Fitness Center,Hockey Arena,Food
5,Davisville Subway Station,Midtown,Coffee Shop,Sushi Restaurant,Pub,Convenience Store,Sandwich Place,Café,Sporting Goods Shop,Middle Eastern Restaurant,Grocery Store,Deli / Bodega,Flower Shop,Moving Target,Spa,Gastropub,Pizza Place
6,Don Mills Subway Station,Suburb,Coffee Shop,Bakery,Women's Store,Tea Room,Smoothie Shop,Jewelry Store,Supplement Shop,Sporting Goods Shop,Boutique,Shopping Mall,Burger Joint,Burrito Place,Frozen Yogurt Shop,Candy Store,Restaurant
7,Donlands Subway Station,Suburb,Pizza Place,Coffee Shop,Café,Beer Bar,Diner,Ramen Restaurant,Restaurant,Convenience Store,Nail Salon,Burger Joint,Gastropub,Sports Bar,Liquor Store,Beer Store,Dim Sum Restaurant
8,Dufferin Subway Station,Suburb,Bakery,Bar,Café,Beer Store,Cocktail Bar,Tea Room,Mexican Restaurant,Flower Shop,Scandinavian Restaurant,Sandwich Place,Coffee Shop,Gift Shop,Fast Food Restaurant,Beer Bar,African Restaurant
9,Dundas West Subway Station,Suburb,Coffee Shop,Bar,Pizza Place,Grocery Store,Pub,Mediterranean Restaurant,Korean Restaurant,Café,Falafel Restaurant,Breakfast Spot,Sporting Goods Shop,Sports Bar,Supermarket,Chinese Restaurant,Sushi Restaurant


10. Cluster locations using K-means 5 clusters

In [14]:
# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Station Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

k_labels = kmeans.labels_

df_merged = stat_df.copy()

# add clustering labels
df_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Station Name'), on='Station Name')

df_merged.head() # check the last columns!

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,High Park Subway Station,43.653926,-79.466868,Suburb,1,Suburb,Convenience Store,Pet Store,Gym,Pharmacy,Coffee Shop,Food Truck,Deli / Bodega,Mexican Restaurant,Pub,Gym Pool,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Royal York Subway Station,43.648119,-79.511381,Suburb,0,Suburb,Coffee Shop,Breakfast Spot,Dessert Shop,Italian Restaurant,Sushi Restaurant,Pub,Restaurant,Seafood Restaurant,Tapas Restaurant,Mobile Phone Shop,Café,French Restaurant,Pool Hall,Gastropub,Pharmacy
3,Victoria Park Subway Station,43.694883,-79.288735,Suburb,0,Suburb,Convenience Store,Golf Course,Bank,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Filipino Restaurant,Event Space,Falafel Restaurant
4,Main Street Subway Station,43.689109,-79.301662,Suburb,0,Suburb,Pharmacy,Moving Target,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant,Fish & Chips Shop,Bakery,Grocery Store,Middle Eastern Restaurant,Pet Store,Sandwich Place,Mexican Restaurant,Coffee Shop,Hockey Arena,Food & Drink Shop


Show on map

In [15]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['lat'], df_merged['lng'], df_merged['Station Name'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [16]:
map_toronto

11. Cluster locations using Agglomerative Clustering 5 clusters

In [17]:
aglo = AgglomerativeClustering(n_clusters=kclusters,linkage='ward').fit(df_grouped_clustering)
aglo.labels_[0:10]
df_merged2 = stat_df.copy()

# add clustering labels
df_merged2['Cluster Labels'] = aglo.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged2 = df_merged2.join(neighborhoods_venues_sorted.set_index('Station Name'), on='Station Name')

df_merged2.head() # check the last columns!

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,1,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,High Park Subway Station,43.653926,-79.466868,Suburb,2,Suburb,Convenience Store,Pet Store,Gym,Pharmacy,Coffee Shop,Food Truck,Deli / Bodega,Mexican Restaurant,Pub,Gym Pool,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Royal York Subway Station,43.648119,-79.511381,Suburb,1,Suburb,Coffee Shop,Breakfast Spot,Dessert Shop,Italian Restaurant,Sushi Restaurant,Pub,Restaurant,Seafood Restaurant,Tapas Restaurant,Mobile Phone Shop,Café,French Restaurant,Pool Hall,Gastropub,Pharmacy
3,Victoria Park Subway Station,43.694883,-79.288735,Suburb,1,Suburb,Convenience Store,Golf Course,Bank,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Filipino Restaurant,Event Space,Falafel Restaurant
4,Main Street Subway Station,43.689109,-79.301662,Suburb,1,Suburb,Pharmacy,Moving Target,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant,Fish & Chips Shop,Bakery,Grocery Store,Middle Eastern Restaurant,Pet Store,Sandwich Place,Mexican Restaurant,Coffee Shop,Hockey Arena,Food & Drink Shop


Show on map

In [18]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged2['lat'], df_merged2['lng'], df_merged2['Station Name'], df_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [19]:
map_toronto

12. Cluster locations using Affinity Propagation

In [20]:
af = AffinityPropagation(preference=-0.35).fit(df_grouped_clustering)
cluster_centers_indices = af.cluster_centers_indices_

n_clusters_ = len(cluster_centers_indices)

df_merged3 = stat_df.copy()

# add clustering labels
df_merged3['Cluster Labels'] = af.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged3 = df_merged3.join(neighborhoods_venues_sorted.set_index('Station Name'), on='Station Name')

df_merged3.head() # check the last columns!

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,High Park Subway Station,43.653926,-79.466868,Suburb,2,Suburb,Convenience Store,Pet Store,Gym,Pharmacy,Coffee Shop,Food Truck,Deli / Bodega,Mexican Restaurant,Pub,Gym Pool,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Royal York Subway Station,43.648119,-79.511381,Suburb,0,Suburb,Coffee Shop,Breakfast Spot,Dessert Shop,Italian Restaurant,Sushi Restaurant,Pub,Restaurant,Seafood Restaurant,Tapas Restaurant,Mobile Phone Shop,Café,French Restaurant,Pool Hall,Gastropub,Pharmacy
3,Victoria Park Subway Station,43.694883,-79.288735,Suburb,0,Suburb,Convenience Store,Golf Course,Bank,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Filipino Restaurant,Event Space,Falafel Restaurant
4,Main Street Subway Station,43.689109,-79.301662,Suburb,0,Suburb,Pharmacy,Moving Target,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant,Fish & Chips Shop,Bakery,Grocery Store,Middle Eastern Restaurant,Pet Store,Sandwich Place,Mexican Restaurant,Coffee Shop,Hockey Arena,Food & Drink Shop


Show on map

In [21]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged3['lat'], df_merged3['lng'], df_merged3['Station Name'], df_merged3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [22]:
map_toronto

13. Cluster locations using Mean Shift clustering

In [23]:
bandwidth = estimate_bandwidth(df_grouped_clustering, quantile=0.2, n_samples=50)

db = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(df_grouped_clustering)

db_labels = db.labels_
#n_clusters_ = (db.core_sample_indices_)

df_merged4 = stat_df.copy()

# add clustering labels
df_merged4['Cluster Labels'] = db.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged4 = df_merged4.join(neighborhoods_venues_sorted.set_index('Station Name'), on='Station Name')

df_merged4.head() # check the last columns!

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,High Park Subway Station,43.653926,-79.466868,Suburb,1,Suburb,Convenience Store,Pet Store,Gym,Pharmacy,Coffee Shop,Food Truck,Deli / Bodega,Mexican Restaurant,Pub,Gym Pool,Food,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
2,Royal York Subway Station,43.648119,-79.511381,Suburb,0,Suburb,Coffee Shop,Breakfast Spot,Dessert Shop,Italian Restaurant,Sushi Restaurant,Pub,Restaurant,Seafood Restaurant,Tapas Restaurant,Mobile Phone Shop,Café,French Restaurant,Pool Hall,Gastropub,Pharmacy
3,Victoria Park Subway Station,43.694883,-79.288735,Suburb,0,Suburb,Convenience Store,Golf Course,Bank,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Filipino Restaurant,Event Space,Falafel Restaurant
4,Main Street Subway Station,43.689109,-79.301662,Suburb,0,Suburb,Pharmacy,Moving Target,Fast Food Restaurant,Fried Chicken Joint,Chinese Restaurant,Fish & Chips Shop,Bakery,Grocery Store,Middle Eastern Restaurant,Pet Store,Sandwich Place,Mexican Restaurant,Coffee Shop,Hockey Arena,Food & Drink Shop


Show on map

In [24]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged4['lat'], df_merged4['lng'], df_merged4['Station Name'], df_merged4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [25]:
map_toronto

14. Function to find stations in each results data frame with is in the cluster with more than 5 members and with required venues: Coffee place, Gym/Fitness, Lunch/fast food and Restaurant

In [26]:
def find_station(df):
    stat_in_cl=[0]*10
    for i in df.index:
        stat_in_cl[df.iloc[i,4]] += 1
    for index, row in df.iterrows():        
        if stat_in_cl[row[4]]>5:
            stat_good=set()
            for j in range(6,16):
                if 'Café' in row[j] or 'Coffee' in row[j]:
                    stat_good.add('coffee')
                if 'Gym' in row[j] or 'Fitness' in row[j] or 'Yoga' in row[j] or 'Pool' in row[j]:
                    stat_good.add('fit')
                if 'Pizza' in row[j] or 'Sandwich' in row[j] or 'Breakfast' in row[j] or 'Salad' in row[j] or 'Burger' in row[j]:
                    stat_good.add('fast')
                if 'Restaurant' in row[j]:
                    stat_good.add('rest')
            if len(stat_good)<4:
                df.drop(index, inplace=True)
        else:
            df.drop(index, inplace=True)
    return(df)

15. Run throuh results of each clustering method and prune data frames to contains only stations meeting the criteria

In [27]:
df_merged=find_station(df_merged)
df_merged.reset_index(inplace=True,drop=True)

df_merged

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,Bloor-Yonge Subway Station,43.670759,-79.385769,Downtown,0,Downtown,Café,Coffee Shop,Gym,Restaurant,Indian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Yoga Studio,Mediterranean Restaurant,French Restaurant,Burger Joint,Deli / Bodega
2,Eglinton West Subway Station,43.699456,-79.436386,Midtown,2,Midtown,Coffee Shop,Record Shop,Bank,Bakery,Convenience Store,Pizza Place,Caribbean Restaurant,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Food,Food Court,Food Truck,Flower Shop,Fish & Chips Shop
3,St. George Subway Station,43.668675,-79.397666,Downtown,0,Downtown,Coffee Shop,Sandwich Place,Restaurant,Cosmetics Shop,Boutique,Gym,Sporting Goods Shop,American Restaurant,Café,Museum,Concert Hall,Breakfast Spot,Bubble Tea Shop,Music School,Shopping Mall
4,Islington Subway Station,43.645097,-79.524429,Suburb,0,Suburb,Coffee Shop,Pizza Place,Sandwich Place,Pub,Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Gym,Convenience Store,Beer Store,Caribbean Restaurant,Bakery,Japanese Restaurant,Dog Run
5,St Clair Subway Station,43.688095,-79.393702,Midtown,0,Midtown,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Pub,Bagel Shop,Pizza Place,Gym,Sandwich Place,Shoe Store,Cantonese Restaurant,Modern European Restaurant,Miscellaneous Shop,Fried Chicken Joint,Convenience Store


In [28]:
df_merged2=find_station(df_merged2)
df_merged2.reset_index(inplace=True,drop=True)

df_merged2

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,1,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,Bloor-Yonge Subway Station,43.670759,-79.385769,Downtown,1,Downtown,Café,Coffee Shop,Gym,Restaurant,Indian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Yoga Studio,Mediterranean Restaurant,French Restaurant,Burger Joint,Deli / Bodega
2,Eglinton West Subway Station,43.699456,-79.436386,Midtown,0,Midtown,Coffee Shop,Record Shop,Bank,Bakery,Convenience Store,Pizza Place,Caribbean Restaurant,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Food,Food Court,Food Truck,Flower Shop,Fish & Chips Shop
3,St. George Subway Station,43.668675,-79.397666,Downtown,1,Downtown,Coffee Shop,Sandwich Place,Restaurant,Cosmetics Shop,Boutique,Gym,Sporting Goods Shop,American Restaurant,Café,Museum,Concert Hall,Breakfast Spot,Bubble Tea Shop,Music School,Shopping Mall
4,Islington Subway Station,43.645097,-79.524429,Suburb,1,Suburb,Coffee Shop,Pizza Place,Sandwich Place,Pub,Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Gym,Convenience Store,Beer Store,Caribbean Restaurant,Bakery,Japanese Restaurant,Dog Run
5,St Clair Subway Station,43.688095,-79.393702,Midtown,1,Midtown,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Pub,Bagel Shop,Pizza Place,Gym,Sandwich Place,Shoe Store,Cantonese Restaurant,Modern European Restaurant,Miscellaneous Shop,Fried Chicken Joint,Convenience Store


In [29]:
df_merged3=find_station(df_merged3)
df_merged3.reset_index(inplace=True,drop=True)

df_merged3

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,Bloor-Yonge Subway Station,43.670759,-79.385769,Downtown,0,Downtown,Café,Coffee Shop,Gym,Restaurant,Indian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Yoga Studio,Mediterranean Restaurant,French Restaurant,Burger Joint,Deli / Bodega
2,Eglinton West Subway Station,43.699456,-79.436386,Midtown,3,Midtown,Coffee Shop,Record Shop,Bank,Bakery,Convenience Store,Pizza Place,Caribbean Restaurant,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Food,Food Court,Food Truck,Flower Shop,Fish & Chips Shop
3,St. George Subway Station,43.668675,-79.397666,Downtown,0,Downtown,Coffee Shop,Sandwich Place,Restaurant,Cosmetics Shop,Boutique,Gym,Sporting Goods Shop,American Restaurant,Café,Museum,Concert Hall,Breakfast Spot,Bubble Tea Shop,Music School,Shopping Mall
4,Islington Subway Station,43.645097,-79.524429,Suburb,0,Suburb,Coffee Shop,Pizza Place,Sandwich Place,Pub,Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Gym,Convenience Store,Beer Store,Caribbean Restaurant,Bakery,Japanese Restaurant,Dog Run
5,St Clair Subway Station,43.688095,-79.393702,Midtown,0,Midtown,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Pub,Bagel Shop,Pizza Place,Gym,Sandwich Place,Shoe Store,Cantonese Restaurant,Modern European Restaurant,Miscellaneous Shop,Fried Chicken Joint,Convenience Store


In [30]:
df_merged4=find_station(df_merged4)
df_merged4.reset_index(inplace=True,drop=True)

df_merged4

,Station Name,lat,lng,Station location,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0,Midtown,Coffee Shop,Fast Food Restaurant,Pub,Italian Restaurant,Thai Restaurant,Liquor Store,Gym,Movie Theater,Asian Restaurant,Salad Place,Supermarket,Smoothie Shop,Bubble Tea Shop,Buffet,Food & Drink Shop
1,Bloor-Yonge Subway Station,43.670759,-79.385769,Downtown,0,Downtown,Café,Coffee Shop,Gym,Restaurant,Indian Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Yoga Studio,Mediterranean Restaurant,French Restaurant,Burger Joint,Deli / Bodega
2,Eglinton West Subway Station,43.699456,-79.436386,Midtown,2,Midtown,Coffee Shop,Record Shop,Bank,Bakery,Convenience Store,Pizza Place,Caribbean Restaurant,Yoga Studio,Food & Drink Shop,Fast Food Restaurant,Food,Food Court,Food Truck,Flower Shop,Fish & Chips Shop
3,St. George Subway Station,43.668675,-79.397666,Downtown,0,Downtown,Coffee Shop,Sandwich Place,Restaurant,Cosmetics Shop,Boutique,Gym,Sporting Goods Shop,American Restaurant,Café,Museum,Concert Hall,Breakfast Spot,Bubble Tea Shop,Music School,Shopping Mall
4,Islington Subway Station,43.645097,-79.524429,Suburb,0,Suburb,Coffee Shop,Pizza Place,Sandwich Place,Pub,Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Gym,Convenience Store,Beer Store,Caribbean Restaurant,Bakery,Japanese Restaurant,Dog Run
5,St Clair Subway Station,43.688095,-79.393702,Midtown,0,Midtown,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Pub,Bagel Shop,Pizza Place,Gym,Sandwich Place,Shoe Store,Cantonese Restaurant,Modern European Restaurant,Miscellaneous Shop,Fried Chicken Joint,Convenience Store


16. Since each output contains the same list of stations the final search can be run on one of them

17. Add distance of each station from Toronto centre location

In [31]:
for i in df_merged.index:
    lat_lon = (df_merged.iloc[i,1],df_merged.iloc[i,2])
    df_merged.loc[i,'Distance'] = geopy.distance.vincenty(Toronto_loc, lat_lon ).km

18. Score each location taking into account:  
    - number of each relevant venues (0.5 point for each Coffee place, 1 for Fitness, 0.5 for lunch/breakfast/fast food, 1 for Restaurant, 0.25 point for other plances)
    - use 1/5 of distance in km to divide score - ie. further from centre lower the score
    - further divide the score by averange cost of office rent per unit of area in each location category - data used from https://www.statista.com/statistics/802500/asking-rent-per-sf-office-space-toronto-by-submarket/ 

In [32]:

for index, row in df_merged.iterrows():        
    stat_good=[]
    for j in range(6,21):
        if 'Café' in row[j] or 'Coffee' in row[j]:
            stat_good.append('coffee')
        if 'Gym' in row[j] or 'Fitness' in row[j] or 'Yoga' in row[j] or 'Pool' in row[j]:
            stat_good.append('fit')
        if 'Pizza' in row[j] or 'Sandwich' in row[j] or 'Breakfast' in row[j] or 'Salad' in row[j] or 'Burger' in row[j] or 'Bakery' in row[j]:
            stat_good.append('fast')
        if 'Restaurant' in row[j] or 'Bar' in row[j] or 'Pub' in row[j]:
            stat_good.append('rest')
    score = (0.5*stat_good.count('coffee'))+stat_good.count('fit')+(0.5*stat_good.count('fast')+stat_good.count('rest'))
    score = score + (15-len(stat_good))*0.25
    score = score/(row['Distance']/0.5)
    if row['Station location']=='Downtown':
        score=score/(30.38*5)    #30.38 average cost of office space in Downtown Toronto
    elif row['Station location']=='Midtown':
        score=score/(23.29*5)    #23.29 average cost of office space in Midtown Toronto
    else:
        score=score/(15.76*5)    #15.76 average cost of office space in Suburbs Toronto
    df_merged.loc[index, 'Score']=score

19. Provide shortlist of 5 best locations ordered by score

In [43]:
df_short = df_merged.nlargest(5,'Score')[['Station Name','lat','lng' ,'Station location', 'Score']]
df_short

,Station Name,lat,lng,Station location,Score
1,Bloor-Yonge Subway Station,43.670759,-79.385769,Downtown,0.021567
3,St. George Subway Station,43.668675,-79.397666,Downtown,0.012526
5,St Clair Subway Station,43.688095,-79.393702,Midtown,0.010374
0,Eglinton Subway Station,43.706716,-79.398298,Midtown,0.006337
4,Islington Subway Station,43.645097,-79.524429,Suburb,0.006137


In [48]:
map_toronto = folium.Map(location=[Toronto_loc[0], Toronto_loc[1]], zoom_start=9)

# set color scheme for the clusters
x = np.arange(5)
ys = [i+x+(i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
i=0
for lat, lon, poi, cluster in zip(df_short['lat'], df_short['lng'], df_short['Station Name'], df_short['Score']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[i-1],
        fill=True,
        fill_color=rainbow[i-1],
        fill_opacity=0.7).add_to(map_toronto)
    i += 1

In [49]:
map_toronto